In [ ]:
## Use case - Tableau subscription function doesn't offer option of PPT output format (image, PDF or both) 
def tableau_ppt_export(name_workbook, output_path, list_of_slides):
    # import all necessary packages
    import sys
    import os
    from datetime import datetime
    from datetime import timedelta
    sys.path.append('/home/user/private/') # to append login details and subprocess function
    from config import ldap_username, ldap_password
    from pptx import Presentation

    # just for file naming purpose
    str_date = datetime.today().strftime("%Y-%m-%d")
    
    # double checking for "/" at the end of the report url to ensure consistent naming
    name_workbook = name_workbook if name_workbook[-1] == '/' else name_workbook+'/'
    name_workbook = '/views/' + name_workbook
    output_path = output_path if output_path[-1] == '/' else output_path+'/'

    # full url including server
    fullurl = f'http://tableau-domain.com/#{name_workbook}'
    
    name_workbook_clean = name_workbook.split('/')[-2]
    
    # Tableau login and download of pngs from the report
    command = 'tabcmd --accepteula'
    accept_eula = run_subprocess_w_output(command, display_output = False) # UDF for calling Tableau server from Python

    command = f'tabcmd login -s http://tableau-domain.com -u {ldap_username} -p {ldap_password} --no-certcheck'
    tableau_login = run_subprocess_w_output(command, display_output = False)
    
    for x in list_of_slides:
        upd_url = str(name_workbook)+x
        command = f'tabcmd get "{upd_url}.png?:refresh=yes" -f "{output_path}{x}.png"'
        get_png = run_subprocess_w_output(command, display_output = False)
        print(get_png)

    command = 'tabcmd logout'
    tableau_logout = run_subprocess_w_output(command, display_output = False)

    # Creating ppt
    presentation_output = Presentation()
   
    for i, m in enumerate (list_of_slides):
        # empty slide
        layout2 = presentation_output.slide_layouts[6]
        # resizing to fit output image
        presentation_output.slide_width = 15240000
        presentation_output.slide_height = 8572500
        #layout2.slide_master.slide_size.set_size(1600, 900, slides.SlideSizeScaleType.DO_NOT_SCALE) 
        slide2 = presentation_output.slides.add_slide(layout2)

        shapes = presentation_output.slides[i].shapes
        picture = shapes.add_picture(output_path+m+'.png',0,0)

    ppt_output_path = output_path+name_workbook_clean+str_date+".pptx"

    presentation_output.save(ppt_output_path)
    
    print("Finished PPT exported in the provided folder")